In [ ]:
import org.apache.spark._
import org.apache.spark.SparkContext._
import org.apache.spark.rdd._
//import scala.io.Source
import scala.collection.JavaConversions._
import scala.util.control.Breaks._
import java.nio.file.{Files, Path, Paths}

object Helpers { // not much required for a script file
    def using[A <: { def close(): Unit }, B](resource: A)(f: A => B): B =
        try {
            f(resource)
        } finally {
            resource.close()
        }
}

import Helpers._

val en : String = "en"
val de : String = "de"
val fr : String = "fr"
val langs : List[String] = List(en, de, fr)

case class Lemma(entry:String, default:String, pos:String)
object Lemma {
  def parse(line:String): Option[Lemma] = {
    line.split("\t") match { 
      case Array(f,s,t,_*) => Some(Lemma(f, s, t))
      case _ => None
    }
  }  
}

// TODO: lemmas per lang : fix
def detectLang(line : String, dicts : scala.collection.mutable.Map[String, List[String]]) : String = {
    line.split(" ").flatMap(item => dicts.filter(_._2.exists(_.equalsIgnoreCase(item))).map(_._1)) // TODO: split not only by " "
                   .groupBy(f => f)
                   .map(g => (g._1, g._2.size))
                   .maxBy(_._2) // TODO: empty (lang not found) exception
                   ._1   
}

val stopwordsPerLang: scala.collection.mutable.Map[String, List[String]] = scala.collection.mutable.Map.empty 
// Stopwords per each lang, are all stored in RAM, because I use them for language detection & they're relatively small, quick to read.  

for(lang <- langs) { // main loop. TODO: refactor into function and call it async
  using(scala.io.Source.fromFile(s"notebooks/words/vocabs/$lang-stopwords.txt")) { source => 
    for (line <- source.getLines) {
      val list = stopwordsPerLang.getOrElse(lang, List.empty)
      stopwordsPerLang.update(lang, list:+line)  
    }
  }
}

val textfilesPathsPerLang: scala.collection.mutable.Map[String, List[String]] = scala.collection.mutable.Map.empty
val textfiles = Files.newDirectoryStream(Paths.get("notebooks/words/text-data"))
  .filter(_.getFileName.toString.endsWith(".txt"))

for(file <- textfiles) {
  using(scala.io.Source.fromFile(file.toString)) { source => 
    val firstLine = source.getLines.next() // get single first line & detect language with it, TODO: better to use a few random lines in the middle of the text
    val lang : String = detectLang(firstLine, stopwordsPerLang)                                                 
    var list = textfilesPathsPerLang.getOrElse(lang, List.empty)
    textfilesPathsPerLang.update(lang, list:+file.toString)                                              
  }
}


//val lemmasPerLang: scala.collection.mutable.Map[String, List[Lemma]] = scala.collection.mutable.Map.empty
for(lang <- langs) { // main loop. TODO: refactor into function and call it async
  
  var lemmas : List[Lemma] = List.empty
  /*
  using(scala.io.Source.fromFile(s"notebooks/words/vocabs/$lang-lemmatizer.txt")) { source => 
    for (line <- source.getLines) {
      val list = lemmasPerLang.getOrElse(lang, List.empty)
      Lemma.parse(line) match { 
        case Some(lemma) => lemmasPerLang.update(lang, list:+lemma)
        case _ =>;
      }      
    }
  }*/
//for(item <- lemmasPerLang.keys) println(item)
//for (item <- stopwordsPerLang.keys ) println(item)
}


import org.apache.spark._
import org.apache.spark.SparkContext._
import org.apache.spark.rdd._
import scala.collection.JavaConversions._
import scala.util.control.Breaks._
import java.nio.file.{Files, Path, Paths}
defined object Helpers
import Helpers._
en: String = en
de: String = de
fr: String = fr
langs: List[String] = List(en, de, fr)
defined class Lemma
defined object Lemma
detectLang: (line: String, dicts: scala.collection.mutable.Map[String,List[String]])String
stopwordsPerLang: scala.collection.mutable.Map[String,List[String]] = Map(en -> List(a, about, above, after, again, against, all, am, an, and, any, are, aren't, as, at, be, because, been, before, being, below, between, both, but, by, can't, cannot, could, couldn't, did, didn't, do, does, doesn't, doing, don't, down, during...